<a href="https://colab.research.google.com/github/yi-ye-zhi-qiu/kitchensoundscapes/blob/main/autoencoder_sparsematrix_input.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


# Autoencoder (sparse matrix input)

In [1]:
from keras.utils import np_utils
from keras.layers import Input, Dense, Flatten, Reshape
from keras import layers
from keras.models import Model, Sequential
from keras.optimizers import RMSprop 

import tensorflow as tf

from music21 import converter, instrument, note, chord, stream
import pickle
import os
import numpy as np

In [2]:
def get_notes(path):

    notes = []

    for filename in os.listdir(path):
          file = path + "/" + filename
          print("Parsing", file)
        
          midi = converter.parse(file)
          need_parse = midi.flat.notes

          for i in need_parse:
            if isinstance(i, note.Note):
                melody.append(str(i.pitch))
            elif isinstance(i, chord.Chord):
                melody.append([str(n) for n in i.pitches])

In [4]:
notes_from_here_please = "/content/drive/My Drive/notebooks/music/training_songs"
notes = get_notes(notes_from_here_please)

Parsing /content/drive/My Drive/notebooks/music/training_songs/breadknife.mid


NameError: ignored

Replace durations (music21 package sometimes gives 7 or 21/3)

In [ ]:
durations = [x.replace('1/3', '0.25') for x in durations]
durations = [x.replace('2/3', '0.75') for x in durations]
durations = [x.replace('4/3', '1') for x in durations]
durations = [x.replace('5/3', '1.25') for x in durations]
durations = [x.replace('7/3', '2.25') for x in durations]
durations = [x.replace('8/3', '2.5') for x in durations]
durations = [x.replace('10/3', '3.25') for x in durations]
durations = [x.replace('13/3', '4.25') for x in durations]
durations = [x.replace('23/3', '7') for x in durations]

Define notes as a string list of each note in melody

In [ ]:
#define notes as a string list of each note in melody 

notes = []
for i, j in enumerate(melody):
  if isinstance(j, list): 
    for k in j:
      notes.append(k)
  else:
    notes.append(j)

def list_to_dict(x): 
    unique = sorted(set([item for item in x]))
    print(len(unique),' unique notes put into dictionary')

    Dict = { i : unique[i] for i in range(0, len(unique) ) }
    print(Dict)

    #Dict = dict((i, j) for j, i in enumerate(x))
    return Dict

notes_d = list_to_dict(notes)

In [ ]:
melody[1:10]

Create sparse matrix

In [ ]:
#initialize sparse matrix
l = []

for i in notes_d.keys():
  l.append([i])
print('Each note has been given a row in matrix l')

# === sparse matrix

def sparse_matrix(l, melody, durations, notes_d):

    print('Melody length is ', len(melody))
    print('Start off with ', l)
    stops, t = {}, 0

    for index, n in enumerate(melody):

        if isinstance(n, list):
            __freeze_time = True
        else:
            __freeze_time = False

        d = dict((k, v) for k, v in stops.items() if v < t)
        #for understanding: print(n, __freeze_time)

        if __freeze_time == True:
            for note in n:
                dur = durations[index]
                m = int(float(dur) / 0.25)
                for i, j in enumerate(l):
                    if note != j[0] and j[0] not in n:
                        for x in range(m):
                            l[i].append(0)
                    else:

                        for x in range(m):
                            l[i].append(1)
                        stops[note] = m

        elif __freeze_time == False:
            dur = durations[index]
            m = int(float(dur) / 0.25)
            #for understanding: print(dur, m, n)
            for i, j in enumerate(l):
                if n != j[0]:
                    for x in range(m):
                        l[i].append(0)
                else:
                    for x in range(m):
                        l[i].append(1)
                    stops[n] = m
        t +=1


    print('Resulting l is', l, '\n we will remove first note or replace it with integer value')
    #prep for sparse matrix by cutting off first note?
    for i in range(len(l)):
        l[i][0] = notes_d[l[i][0]]

    #might need to preserve note order for later..
    # order_ = []
    # for i in range(len(l)):
    #     order_.append(l[i][0])
    return l

l = sparse_matrix(l, melody, durations, notes_d)

from scipy.sparse import csr_matrix, csr_matrix

import sys 
#print(np.array(l))
#this doesn't work for autoencoder: sparse = csc_matrix(l)
sparse = csr_matrix(l, dtype=np.int8).toarray()
print('Sparse matrix of form m x n where m=note and n=duration stored in l')
print("Memory utilised (bytes): ", sys.getsizeof(sparse))


Notice that sparse is now sorted by col=0 (if note kept),

In [ ]:
print(sparse.shape)
sparse[0:2]

# Run *autoencoder*

## Seeding & hyperparameters
- Seed as to not get random result when training
- Hyperparameters:
  - sequence_length: how long your output song will be;
  - input_sample: # of unique notes (rows in matrix);
  - input_notes: how many notes you have in total (cols in matrix);
  - input_dim: same as input_notes;
  - latent_dim: the # of dimensions we "scrunch down to";
  - epochs: # of epochs;
  - learning_rate: learning rate.

In [ ]:
""" Seeding """
np.random.seed(42)
tf.random.set_seed(42)

""" Hyperparameters """
#Notes_ = Notes_[1:2000]
Notes_ = np.array(sparse)
print('Input shape: ', Notes_.shape)
sequence_length = 1

input_sample = Notes_.shape[0]
print('input_sample (datasize) is the # of unique notes (# of rows in sparse matrix), ', input_sample)
input_notes = Notes_.shape[1]
input_dim = input_notes
print('input_dim (# of cols in matrix, synonomous w/ length of song) are', input_dim)

latent_dim = 2

epochs = 200

learning_rate = 0.000013

Run model

Encoder

In [ ]:
##Encoder
#EncInput = Input(shape = (number_of_samples))
EncInput = Input(shape= (input_dim,))
#shrink down to shape of latent dim
Enc = Dense(latent_dim, activation = 'sigmoid')(EncInput)
print('Encoder',Enc)
encode = Model(EncInput, Enc)

Decoder

In [ ]:
##Decoder
#start @ shape of latent dim
DecInput = Input(shape= (latent_dim,))
#go back out to input_dim
Dec = Dense(input_dim, activation = 'sigmoid')(DecInput)
print('Decoder',Dec)
decode = Model(DecInput, Dec)

Autoencoder

In [ ]:
##Autoencoder
autoencoder = Model(EncInput, decode(Enc))

#Try different learning rates, batch sizes, dropout layer (?), add another layer (if more data)? 
#autoencoder.compile(loss = 'binary_crossentropy', optimizer=RMSprop(learning_rate=0.00013))
autoencoder.compile(loss = 'binary_crossentropy', optimizer='adam')

# Generation

Approach: use a random "starting point" & decode that

In [ ]:
D_MEl = decode([np.random.normal(size =(1, latent_dim))]).numpy().reshape(input_notes) 

In [ ]:
print('durations?', D_MEl)
print('nonono want a 48 x 3809 shape honestly you"ve gotta be joshin me...')

# == reconstruction of song ... == #

prepped = [round(x*4)/4 for x in D_MEl]
print(prepped)

def find_key(input_dict, value):
    return next((k for k, v in input_dict.items() if v == value), None)

__arr = []
for i in range(78): 
  q = find_key(notes_d, i)
  #__durs = []
  # for j in prepped:
  #   d = duration.Duration(j)
  __arr.append([note.Note(q, quarterLength=j) for j in prepped])
  #   __arr.append([note.Note(q, duration=d)])
print('single melody created in __arr')
print(__arr[1:10][1:10])

In [ ]:
print('inputting into stream object, s_obj ... \n')
s_obj = stream.Stream()
s_obj.append(instrument.Piano())

for i in __arr[1:10][1:10]: #for each note
    print([y.duration.quarterLength for y in i])
    print(len(i), i)
    #take the "stack" of notes as psuedo-chords
    s_obj.append(chord.Chord([j for j in i]))

print('input complete, generating as "generated.mid"...')
s_obj.write('midi', fp='/content/drive/My Drive/notebooks/music'+'/generated.mid')